In [ ]:

from mat3ra.made.material import Material
from utils.jupyterlite import get_data
from utils.visualize import visualize_materials


get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))
material = materials[1]
visualize_materials( material, repetitions=[3,3,1])

In [ ]:
import numpy as np

def create_matrices(max_search: int):
    matrices = []
    for s11 in range(-max_search, max_search + 1):
        for s12 in range(-max_search, max_search + 1):
            for s21 in range(-max_search, max_search + 1):
                for s22 in range(-max_search, max_search + 1):
                    # Non-zero area constraint
                    matrix = np.array([[s11, s12], [s21, s22]])
                    determinant = np.linalg.det(matrix)
                    # If matrices are degenerate or contain mirroring, skip
                    if determinant == 0 or determinant < 0:
                        continue
                    matrices.append(matrix)
    return matrices
matrices = create_matrices(1)
print(len(matrices))

In [ ]:
# Solve for specific rotation matrix
angle = np.pi/2
R = [[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]]
print("R matrix = ", R)
from mat3ra.made.tools.build.supercell import create_supercell
commensurate_lattices = []
for index1, matrix1 in enumerate(matrices):
    for index2, matrix2 in enumerate(matrices[0:index1+1]):
        # print(index1, index2)
        # print("m1@m2\n", np.linalg.inv(matrix2) @ matrix1)
        matrix2_inverse = np.linalg.inv(matrix2) 
        product = matrix2_inverse @ matrix1
        # print(product)
        # is_eye_matrix1 = np.allclose(matrix1, np.eye(2), atol=0.01)
        # is_eye_matrix2 = np.allclose(matrix2, np.eye(2), atol=0.01)
        # if is_eye_matrix1 and is_eye_matrix2:
        #     print("eye matrix")
        are_close = np.allclose(product, R, atol=0.01)
        if  are_close:                  
            commensurate_lattices.append((matrix1, matrix2))
print("all commensurate matrices:", len(commensurate_lattices))

# commensurate_lattices[1]

In [ ]:
material.lattice.vector_arrays

In [ ]:
def solve_angle_from_rotation_matrix(matrix, zero_tolerance=1e-6, round_digits=3):
    if matrix.shape != (2, 2):
        raise ValueError("Input matrix must be 2x2")
    if np.abs(np.linalg.det(matrix) - 1) > zero_tolerance:
        raise ValueError("Matrix must be orthogonal (determinant = 1)")
    if not np.all(np.abs(matrix) <= 1):
        raise ValueError("Matrix have all elements less than 1")
    
    # Extract the elements of the matrix
    cos_theta = matrix[0, 0]
    sin_theta = matrix[1, 0]

    # Calculate the angle in radians
    angle_rad = np.arctan2(sin_theta, cos_theta)

    # Convert the angle to degrees
    angle_deg = np.round(np.degrees(angle_rad), round_digits)

    return angle_deg

# Solve for all angles
# angle = np.pi/2
# R = [[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]]
max_search = 3
matrices = create_matrices(max_search)
a1 = [2.467291, 0.0]
a2 = [-1.2336454999999995, 2.136736684528712]
matrix_a1a2 = np.array([a1, a2])
matrix_a1a2_inverse = np.linalg.inv(matrix_a1a2)

from mat3ra.made.tools.build.supercell import create_supercell
commensurate_lattices = []
angles = []
determinants = []

solutions = []
for index1, matrix1 in enumerate(matrices):
    for index2, matrix2 in enumerate(matrices[0:index1+1]):
        # print(index1, index2)
        # print("m1@m2\n", np.linalg.inv(matrix2) @ matrix1)
        matrix2_inverse = np.linalg.inv(matrix2)
        intermediate_product = matrix2_inverse @ matrix1
        product = matrix_a1a2_inverse @ intermediate_product @ matrix_a1a2
        try:
            angle = solve_angle_from_rotation_matrix(product)
            # print(matrix1, matrix2, angle)
            commensurate_lattices.append((matrix1, matrix2, angle))
            angles.append(angle)
            size_metric = np.linalg.det(matrix_a1a2_inverse @ matrix1 @ matrix_a1a2)
            solutions.append({"matrix1": matrix1, "matrix2": matrix2, "angle": angle, "size_metric": size_metric})
        except ValueError:
            # print("Error in calculating angle")
            continue
        # print(product)
print("all commensurate matrices:", len(commensurate_lattices))
angles.sort()
unique_angles = sorted(set(angles))
print("number of angles:", len(unique_angles))
print(unique_angles)

In [ ]:
# Assuming `solutions` is a list of dictionaries with an "angle" key
unique_solutions = {}
for solution in solutions:
    angle = solution["angle"]
    if angle not in unique_solutions:
        unique_solutions[angle] = solution

filtered_solutions = list(unique_solutions.values())
sorted_filtered_solutions = sorted(filtered_solutions, key=lambda x: x["angle"])
import matplotlib.pyplot as plt
import mplcursors

# Assuming `sorted_filtered_solutions` is already defined
angles = [sol["angle"] for sol in sorted_filtered_solutions]
determinants = [sol["size_metric"] for sol in sorted_filtered_solutions]
from typing import List, Dict, Union
import plotly.graph_objs as go

def plot_angles_vs_size_metric(solutions: List[Dict[str, Union[str, int]]], settings: Dict[str, Union[str, int]]):
    """
    Plot the angles vs size_metric for each solution.
    Allows for visual inspection to find the most optimal solution for the current task.
    
    Args:
        solutions (List[Dict[str, Union[str, int]]]): The solutions to plot.
        settings (Dict[str, Union[str, int]]): The settings for the plot.
    """

    data = []
    for index, solution in enumerate(solutions):
        angle = solution["angle"]
        size_metric = solution["size_metric"]

        hover_text = f"Index: {index}<br>Angle: {angle:.2f}°<br>Size Metric: {size_metric:.2f} <br>Matrix1: {solution['matrix1']} <br>Matrix2: {solution['matrix2']}"

        trace = go.Scatter(
            x=[angle],
            y=[size_metric],
            text=[hover_text],
            mode="markers",
            hoverinfo="text",
            name=f"Index: {index}",
        )
        data.append(trace)

    layout = go.Layout(
        xaxis=dict(title="Angle (°)", type=settings["X_SCALE"]),
        yaxis=dict(title="Size Metric", type=settings["Y_SCALE"]),
        hovermode="closest",
        height=settings["HEIGHT"],
        legend_title_text="Solution Indices",
    )

    fig = go.Figure(data=data, layout=layout)
    fig.show()
    
    
# Define the settings for the plot
settings = {
    "X_SCALE": "linear",
    "Y_SCALE": "linear",
    "HEIGHT": 800,
}

# Plot the angles vs size_metric
plot_angles_vs_size_metric(sorted_filtered_solutions, settings)
